In [1]:
from data import update_expl_data, update_target_data, food_categories, preprocess_expl, load_statscan
import pandas as pd
import os
from dotenv import load_dotenv

In [2]:
"""
Set sample rate. In this notebook, all data will be resampled at the chosen frequency.
'MS' : Monthly (Month Start)
'W' : Weekly
'D' : Daily
"""

frequency = 'MS'

In [3]:
"""
Load food CPI data from January 1986 to the most recently available data.
"""
if not os.path.exists("./data_files"):
    os.mkdir("./data_files")
if not os.path.exists("./statscan_data"):
    os.mkdir("./statscan_data")
foodprice_df = update_target_data(food_categories, './data_files/food_cpi.csv')
foodprice_df = foodprice_df.resample(frequency).mean().interpolate()
foodprice_df

18100004-eng.zip: 100%|██████████| 14.3M/14.3M [00:04<00:00, 3.05MB/s]


,Bakery and cereal products (excluding baby food),Dairy products and eggs,"Fish, seafood and other marine products",Food purchased from restaurants,Food,"Fruit, fruit preparations and nuts",Meat,Other food products and non-alcoholic beverages,Vegetables and vegetable preparations
REF_DATE,,,,,,,,,
1986-01-01,69.3,70.9,60.6,59.1,67.3,76.0,65.1,77.5,76.0
1986-02-01,70.3,70.8,61.3,59.1,66.9,77.6,64.2,78.1,68.4
1986-03-01,70.6,71.1,61.3,59.3,67.0,79.2,64.2,78.6,66.2
1986-04-01,71.3,71.0,61.4,59.7,67.7,82.2,63.6,79.5,71.1
1986-05-01,71.2,71.4,61.9,59.9,68.2,83.5,64.0,79.8,75.3
...,...,...,...,...,...,...,...,...,...
2023-05-01,198.9,170.2,166.5,186.4,184.6,168.6,203.2,173.5,185.6
2023-06-01,197.7,169.8,166.0,187.0,184.8,172.4,204.0,173.0,183.0
2023-07-01,199.3,170.8,164.3,188.1,185.5,166.6,206.7,173.8,185.2


In [7]:
if not os.path.exists("./datasets"):
    os.mkdir("./datasets")
foodprice_df.to_csv("./datasets/food_CPI.csv")

### Get energy CPI from StatCan as well

In [8]:
"""
Load energy CPI data from January 1986 to the most recently available data.
"""
energy_df = load_statscan(['Energy'])
energy_df = energy_df.resample(frequency).mean().interpolate()
energy_df

,Energy
REF_DATE,
1986-01-01,69.4
1986-02-01,69.4
1986-03-01,67.1
1986-04-01,62.6
1986-05-01,61.0
...,...
2023-05-01,204.4
2023-06-01,207.2
2023-07-01,210.7


### FRED

In [12]:
fred_data_sources = list(pd.read_csv("./FRED_series_names.csv")['0'].unique())
fred_data_sources

['A229RX0',
 'AAA',
 'AAA10YM',
 'AHETPI',
 'ALTSALES',
 'AMBNS',
 'AMBSL',
 'BAA',
 'BAA10YM',
 'BOGMBASE',
 'BUSLOANS',
 'BUSLOANSNSA',
 'CANCP010000GPM',
 'CANCP010000GYM',
 'CANCP010000IXNBM',
 'CANCP010000IXOBM',
 'CANCP040100GPM',
 'CANCP040100GYM',
 'CANCP040100IXOBM',
 'CANCP040200GPM',
 'CANCP040200GYM',
 'CANCP040200IXOBM',
 'CANCPALTT01IXNBM',
 'CANCPGRGO01GPM',
 'CANCPGRGO01GYM',
 'CANCPGRGO01IXOBM',
 'CANCPGRHO01GYM',
 'CANCPGRHO01IXOBM',
 'CANCPGRLH01GPM',
 'CANCPGRLH01GYM',
 'CANCPGRLH01IXOBM',
 'CANCPGRSE01GPM',
 'CANCPGRSE01GYM',
 'CANCPGRSE01IXOBM',
 'CANCPIALLMINMEI',
 'CANCPICORMINMEI',
 'CANCPIENGMINMEI',
 'CANEPUINDXM',
 'CANHOUREAMISMEI',
 'CANLOCOBSNOSTSAM',
 'CANLOCOBSORSTSAM',
 'CANLOCOISNOSTSAM',
 'CANLOCOISORSTSAM',
 'CANLOCOMANOSTSAM',
 'CANLOCOSINOSTSAM',
 'CANLOCOSIORSTM',
 'CANLOCOSPNOSTSAM',
 'CANLOCOSPORIXOBM',
 'CANLOLITOAASTSAM',
 'CANLOLITONOSTSAM',
 'CANLOLITOTRGYSAM',
 'CANLOLITOTRSTSAM',
 'CANLORSGPNOSTSAM',
 'CANLORSGPRTSTSAM',
 'CANLORSGPTDSTSA

In [13]:
"""
Load exogenous/auxiliary explanatory variables from FRED: https://fred.stlouisfed.org/
These data sources reflect various economic factors that may improve forecasts. 
Please visit the FRED website to learn more about these series, and to find others
that may be useful for food CPI forecasting. 
"""

load_dotenv("../.env")
FRED_API_KEY = os.getenv("FRED_API_KEY")

data_sources = fred_data_sources

expl_df = preprocess_expl(update_expl_data(data_sources, './data_files/expl_vars.csv', sleep_sec=0.5, api_key=FRED_API_KEY))
expl_df_monthly = expl_df.resample(frequency).mean().interpolate()
expl_df_monthly

,A229RX0,AAA,AAA10YM,AHETPI,ALTSALES,AMBNS,AMBSL,BAA,BAA10YM,BOGMBASE,...,XTIMVA01CAM657S,XTIMVA01CAM659S,XTIMVA01CAM664N,XTIMVA01CAM664S,XTIMVA01CAM667S,XTNTVA01CAM664N,XTNTVA01CAM664S,XTNTVA01CAM667S,TOTALNS,TOTALSL
1986-01-01,26317.0,10.05,0.86,8.85,15.781,211.776,210.474,11.44,2.25,221700.0,...,3.644028,12.050836,9.368000e+09,9.496200e+09,6.748621e+09,6.906000e+08,1.052400e+09,7.479043e+08,607.36898,605.70302
1986-02-01,26439.0,9.67,0.97,8.88,15.126,207.917,211.126,11.11,2.41,217800.0,...,1.964888,16.745358,9.495000e+09,9.632200e+09,6.881224e+09,-9.880000e+07,1.539000e+08,1.099459e+08,605.80702,610.67829
1986-03-01,26692.0,9.00,1.22,8.89,13.900,211.716,213.741,10.50,2.72,221400.0,...,-11.565073,1.654991,8.802600e+09,8.528600e+09,6.085406e+09,9.138000e+08,9.079000e+08,6.478132e+08,606.79900,613.37662
1986-04-01,26773.0,8.79,1.49,8.88,15.579,214.529,215.304,10.19,2.89,225100.0,...,13.334090,10.821208,1.034410e+10,9.568800e+09,6.896839e+09,3.470000e+08,6.563000e+08,4.730369e+08,614.36685,619.65773
1986-05-01,26775.0,9.09,1.38,8.90,16.038,216.585,217.379,10.29,2.58,226400.0,...,-4.235546,6.159794,9.597500e+09,9.090800e+09,6.604720e+09,7.013000e+08,6.893000e+08,5.007957e+08,621.91515,625.81996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-01,50236.0,4.67,1.10,28.72,15.514,3333.482,3330.788,5.77,2.20,5569600.0,...,3.042468,-2.672688,6.668240e+10,6.355760e+10,4.701218e+10,-6.519000e+08,-9.628000e+08,-7.121622e+08,4917.78949,4960.00115
2023-06-01,50194.0,4.65,0.90,28.82,16.054,3333.482,3330.788,5.75,2.00,5608500.0,...,1.487421,-5.404119,6.611770e+10,6.339810e+10,4.771144e+10,-3.931400e+09,-3.536300e+09,-2.661310e+09,4942.92720,4973.79132
2023-07-01,50051.0,4.66,0.76,28.94,15.938,3333.482,3330.788,5.74,1.84,5517800.0,...,-4.237659,-7.247550,5.900850e+10,6.038250e+10,4.568960e+10,-9.741000e+08,8.890000e+07,6.726792e+07,4947.46213,4984.78069
2023-08-01,49969.0,4.95,0.78,29.00,15.354,3333.482,3330.788,6.02,1.85,5559100.0,...,-4.237659,-7.247550,5.900850e+10,6.038250e+10,4.568960e+10,-9.741000e+08,8.890000e+07,6.726792e+07,4950.83912,4969.15298


In [14]:
combined_df = pd.concat((foodprice_df, energy_df, expl_df_monthly,), axis=1).dropna(axis=0)
combined_df.to_csv("./datasets/all_data.csv")
combined_df

,Bakery and cereal products (excluding baby food),Dairy products and eggs,"Fish, seafood and other marine products",Food purchased from restaurants,Food,"Fruit, fruit preparations and nuts",Meat,Other food products and non-alcoholic beverages,Vegetables and vegetable preparations,Energy,...,XTIMVA01CAM657S,XTIMVA01CAM659S,XTIMVA01CAM664N,XTIMVA01CAM664S,XTIMVA01CAM667S,XTNTVA01CAM664N,XTNTVA01CAM664S,XTNTVA01CAM667S,TOTALNS,TOTALSL
1986-01-01,69.3,70.9,60.6,59.1,67.3,76.0,65.1,77.5,76.0,69.4,...,3.644028,12.050836,9.368000e+09,9.496200e+09,6.748621e+09,6.906000e+08,1.052400e+09,7.479043e+08,607.36898,605.70302
1986-02-01,70.3,70.8,61.3,59.1,66.9,77.6,64.2,78.1,68.4,69.4,...,1.964888,16.745358,9.495000e+09,9.632200e+09,6.881224e+09,-9.880000e+07,1.539000e+08,1.099459e+08,605.80702,610.67829
1986-03-01,70.6,71.1,61.3,59.3,67.0,79.2,64.2,78.6,66.2,67.1,...,-11.565073,1.654991,8.802600e+09,8.528600e+09,6.085406e+09,9.138000e+08,9.079000e+08,6.478132e+08,606.79900,613.37662
1986-04-01,71.3,71.0,61.4,59.7,67.7,82.2,63.6,79.5,71.1,62.6,...,13.334090,10.821208,1.034410e+10,9.568800e+09,6.896839e+09,3.470000e+08,6.563000e+08,4.730369e+08,614.36685,619.65773
1986-05-01,71.2,71.4,61.9,59.9,68.2,83.5,64.0,79.8,75.3,61.0,...,-4.235546,6.159794,9.597500e+09,9.090800e+09,6.604720e+09,7.013000e+08,6.893000e+08,5.007957e+08,621.91515,625.81996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-01,198.9,170.2,166.5,186.4,184.6,168.6,203.2,173.5,185.6,204.4,...,3.042468,-2.672688,6.668240e+10,6.355760e+10,4.701218e+10,-6.519000e+08,-9.628000e+08,-7.121622e+08,4917.78949,4960.00115
2023-06-01,197.7,169.8,166.0,187.0,184.8,172.4,204.0,173.0,183.0,207.2,...,1.487421,-5.404119,6.611770e+10,6.339810e+10,4.771144e+10,-3.931400e+09,-3.536300e+09,-2.661310e+09,4942.92720,4973.79132
2023-07-01,199.3,170.8,164.3,188.1,185.5,166.6,206.7,173.8,185.2,210.7,...,-4.237659,-7.247550,5.900850e+10,6.038250e+10,4.568960e+10,-9.741000e+08,8.890000e+07,6.726792e+07,4947.46213,4984.78069
2023-08-01,197.9,168.8,165.1,189.0,185.3,164.6,206.6,174.9,182.5,218.4,...,-4.237659,-7.247550,5.900850e+10,6.038250e+10,4.568960e+10,-9.741000e+08,8.890000e+07,6.726792e+07,4950.83912,4969.15298
